<a href="https://colab.research.google.com/github/hasini-06/Gen-AI---Assignment2/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task - 1**

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import InferenceClient
from getpass import getpass
import textwrap

HF_TOKEN = getpass("Enter the token : ")

client=InferenceClient(model="deepset/roberta-base-squad2",token=HF_TOKEN)

question=input("Enter the query : ")

context="The book was written by J.K. Rowling in 1997."

try:
  response=client.question_answering(context=context,question=question)
  answer=response.get("answer","answer not found")
  print(textwrap.fill(answer,width=50))

except Exception as e:
  print("Error please try again!!!")
  print("Details:",e)

Enter the token : ··········
Enter the query : Who wrote the book
J.K. Rowling


In [4]:
!pip install transformers
from transformers import pipeline
from huggingface_hub import InferenceClient
import traceback
from getpass import getpass

#Step-1:  Try loading GPT-2 locally

try:
  local_generator=pipeline("text-generation",model="gpt2")
except Exception as e:
  print("Failed to load GPT-2 locally:",e)
  local_generator=None

#Step - 2: Set up Hugging Face API Client
HF_API_TOKEN = getpass("Enter the token : ")
client=InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    token=HF_API_TOKEN
)

#Step - 3: Fallback logic
def generate_text(prompt):
  try:
    print("Trying GPT-2 locally....")
    if local_generator:
      result=local_generator(prompt,max_length=50)[0]["generated_text"]
      return f"GPT-2 (local):\n{result}"
    else:
      raise RuntimeError("GPT-2 not available")
  except Exception as e:
    print("Local GPT-2 failed. Failing back to Hugging face Inference API...")
    try:
      messages = [
          {"role": "system", "content":"You are a helpful assistant."},
          {"role": "user","content": prompt}
      ]
      response = client.chat_completion(messages=messages, max_tokens = 50)
      result = response.choices[0].message.content.strip()
      return f"Hugging Face API: \n{result}"
    except Exception as api_error:
      return f"Both methods failed. \nError:\n{traceback.format_exc()}"   #returns error in string format

#Step-4 : prompt input and run
user_input = input("Enter your prompt: ")
output = generate_text(user_input)
print("\n" + output)



Device set to use cpu


Enter the token : ··········
Enter your prompt: What are the uses of cycling


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Trying GPT-2 locally....

GPT-2 (local):
What are the uses of cycling?

It has been estimated that cycling in Australia contributes about $5 billion to the economy annually, a number that is expected to grow. A recent estimate by the Australian Institute of Sport suggests that cycling contributes to about $1 billion in economic activity.

However, in 2011 the Australian Institute of Sport (AIS) published a report that found that cycling is one of the most important activities for human wellbeing and wellbeing.

The AIS report found that cycling accounted for a significant proportion of the Australian economy in 2011-12.

How did cycling impact Australia?

Carrying a bike in Australia was not as often as people think. However, people may not have realised that cycling is one of the most important activities for their wellbeing.

While cycling is a very popular activity in Australia, its impact on society is limited.

There is no evidence that cycling causes serious social or economic da

**Task - 2**

In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import textwrap


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")   #GPT-2 Tokenzier
model = GPT2LMHeadModel.from_pretrained("gpt2")   #GPT-2 Model


pad_token_id = tokenizer.eos_token_id   #Pad token to avoids warnings
model.config.pad_token_id = pad_token_id    #Updates the model config to use the eos_token as the pad token.


def generate_story(prompt, temperature=0.7):
    """
    Generates a story based on a given prompt and temperature.

    Args:
        prompt (str): The input text to continue.
        temperature (float): Controls randomness of output (higher = more random).

    Returns:
        str: Generated story text.
    """
    try:

        input_ids = tokenizer.encode(prompt, return_tensors="pt")   #Changes iput to ID's
        output_ids = model.generate(
            input_ids,
            max_length=150,
            do_sample=True,
            top_k=50,   #Considers only the top 50 most likely next tokens.
            top_p=0.95,   #chooses tokens from top 95% of cumulative probability.
            temperature=temperature,
            repetition_penalty=1.2,   #Penalizes repeating the same phrases to make text more diverse
            pad_token_id=pad_token_id  # avoid padding warning
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)   # Decode and return the story
        return story

    except Exception as e:
        print("Error generating story:", e)   #Error-handling
        return None


prompt="Once upon a time there lived a dragon"
temperature=[0.5,0.7,0.9,1.3]

for i in temperature:
  story=generate_story(prompt,i)    #function calling
  print("temperature : "+str(i))
  print(textwrap.fill(story,width=80))
  print("\n")


temperature : 0.5
Once upon a time there lived a dragon who was not of the same race as you. "You
are my friend, I trust your judgment." The Dragon said in his voice and then
turned to face me again with that smile on her lips she still had it now but
this one didn't look like anything else than something very familiar even though
he knew what kind they were at least for him because if nothing more could be
done about them all would just vanish into thin air… "I'm sorry…" She whispered
coldly before turning back around once we'd left our room together after talking
some good old fashioned game play while looking up from their bookshelf trying
desperately hard against any urge or emotion which might have been causing such
confusion amongst us so


temperature : 0.7
Once upon a time there lived a dragon, who was known as the Dragon King of
Rohan. The man's name is Valar and he held an immense fortune in this
world...but it wasn't enough to get him into Imperial hands; she also had his
bl